In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3" 
import torch
torch.cuda.device_count()

1

In [2]:
import os
import shutil
import tempfile

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from glob import glob
# import wandb

import monai
from monai.losses import DiceCELoss, DiceFocalLoss, FocalLoss
from monai.inferers import sliding_window_inference
from monai import transforms

from monai.transforms import (
       AsDiscrete,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandFlipd,
    RandCropByPosNegLabeld,
    RandShiftIntensityd,
    ScaleIntensityRanged,
    Spacingd,
    RandRotate90d,
    MapTransform,
    ScaleIntensityd,
    #AddChanneld,
    SpatialPadd,
    CenterSpatialCropd,
    EnsureChannelFirstd,
    ConcatItemsd,
    AdjustContrastd, 
    Rand3DElasticd,
    HistogramNormalized,
    NormalizeIntensityd,
    Invertd,
    SaveImage,

)

from monai.config import print_config
from monai.metrics import DiceMetric
from monai.networks.nets import SwinUNETR, UNETR, SegResNet
from monai.data import (
    DataLoader,
    CacheDataset,
    load_decathlon_datalist,
    decollate_batch,
)
from monai import data


from monai.utils import first, set_determinism
from sklearn.model_selection import train_test_split
import json


import torch

In [3]:
set_determinism(seed=1024)

In [11]:
import os
import json
import random
from glob import glob

def generate_paths(patient_id):
    base_dir = '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data'
    return {
        'id': os.path.join(base_dir, patient_id),
        'ct': os.path.join(base_dir, patient_id, f"{patient_id}_ct.nii.gz"),
        'seg': os.path.join(base_dir, patient_id, f"{patient_id}_gt.nii.gz")
    }

# Assign each data entry to a random fold
all_data = []
num_folds = 5

for file_dir in sorted(glob('data/*')):
    patient_id = file_dir.split('/')[-1]
    # Check if the file belongs to the MDA center
    if patient_id.startswith(("CHUM", "CHUS", "CHUV","CHUP" ,"MDA")):
        entry = generate_paths(patient_id)
        entry['fold'] = random.randint(1, num_folds) - 1
        all_data.append(entry)

# Compile data into a JSON structure
data_json = {"training": all_data}

# Save to JSON file
json_file_path = "/home/nada.saadi/CTPET/hecktor2022_cropped/5centers_train_data.json"
with open(json_file_path, 'w') as f:
    json.dump(data_json, f, indent=4)

print(f"JSON file created at {json_file_path}")


JSON file created at /home/nada.saadi/CTPET/hecktor2022_cropped/5centers_train_data.json


In [12]:
import json

json_file_path = "/home/nada.saadi/CTPET/hecktor2022_cropped/5centers_train_data.json"

with open(json_file_path, 'r') as f:
    data = json.load(f)

first_5_elements = data["training"][:5]
print(first_5_elements)

[{'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/CHUM-001', 'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/CHUM-001/CHUM-001_ct.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/CHUM-001/CHUM-001_gt.nii.gz', 'fold': 2}, {'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/CHUM-002', 'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/CHUM-002/CHUM-002_ct.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/CHUM-002/CHUM-002_gt.nii.gz', 'fold': 1}, {'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/CHUM-006', 'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/CHUM-006/CHUM-006_ct.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/CHUM-006/CHUM-006_gt.nii.gz', 'fold': 0}, {'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/CHUM-007', 'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/CHUM-007/CHUM-007_ct.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/CHUM-007/CHUM-007_gt.nii.gz',

In [13]:
data_dir = '/home/nada.saadi/MIS-FM/hecktor2022_cropped'
json_dir = "/home/nada.saadi/CTPET/hecktor2022_cropped/5centers_train_data.json"

In [14]:
def datafold_read(datalist, basedir, fold=0, key="training"):
    with open(datalist) as f:
        json_data = json.load(f)

    json_data = json_data[key]

    for d in json_data:
        for k in d:
            if isinstance(d[k], list):
                d[k] = [os.path.join(basedir, iv) for iv in d[k]]
            elif isinstance(d[k], str):
                d[k] = os.path.join(basedir, d[k]) if len(d[k]) > 0 else d[k]

    tr = []
    val = []
    for d in json_data:
        if "fold" in d and d["fold"] == fold:
            val.append(d)
        else:
            tr.append(d)

    return tr, val

In [15]:
train_files, validation_files = datafold_read(datalist=json_dir, basedir=data_dir, fold=0)
len(train_files), len(validation_files)

(370, 79)

In [16]:
class ClipCT(MapTransform):
    """
    Convert labels to multi channels based on hecktor classes:
    label 1 is the tumor
    label 2 is the lymph node

    """

    def __call__(self, data):
        d = dict(data)
        for key in self.keys:
            if key == "ct":
                d[key] = torch.clip(d[key], min=-200, max=200)
            # elif key == "pt":
            #     d[key] = torch.clip(d[key], d[key].min(), 5)
        return d

class MulPTFM(MapTransform):
    """
    Mult PT and FM 

    """

    def __call__(self, data):
        d = dict(data)

        fm = d["ct"] > 0
        d["pt"] = d["pt"] * fm
        return d

class SelectClass(MapTransform):
    """
    Select the class for which you want to fine tune the model 

    """
    # def __init__(self, keys, cls=1):
    #     super(self).__init__(keys)
    #     self.cls = cls

    def __call__(self, data):
        d = dict(data)
        d["seg"][d["seg"] == 1] = 0
        # d["seg"][d["seg"] == 2] = 1
        
        return d

In [17]:
num_samples = 4

train_transforms = Compose(
    [
        LoadImaged(keys=["ct", "seg"], ensure_channel_first = True),
        SpatialPadd(keys=["ct",  "seg"], spatial_size=(200, 200, 310), method='end'),
        Orientationd(keys=["ct",  "seg"], axcodes="PLS"),
        #NormalizeIntensityd(keys=["pt"]),
        ClipCT(keys=["ct"]),
        ScaleIntensityd(keys=["ct"], minv=0, maxv=1),
        #MulPTFM(keys=["ct","pt"]),
        #ConcatItemsd(keys=["pt", "ct"], name="ctpt"),
        #NormalizeIntensityd(keys=["ctpt"], channel_wise=True),
        RandCropByPosNegLabeld(
            keys=["ct", "seg"],
            label_key="seg",
            spatial_size=(96, 96, 96),
            pos=1,
            neg=1,
            num_samples=num_samples,
            image_key="ct",
            image_threshold=0,
        ),
        RandFlipd(
            keys=["ct", "seg"],
            spatial_axis=[0],
            prob=0.20,
        ),
        RandFlipd(
            keys=["ct", "seg"],
            spatial_axis=[1],
            prob=0.20,
        ),
        RandFlipd(
            keys=["ct", "seg"],
            spatial_axis=[2],
            prob=0.20,
        ),
        RandRotate90d(
            keys=["ct", "seg"],
            prob=0.20,
            max_k=3,
        ),
    ]
)
val_transforms = Compose(
    [
        LoadImaged(keys=["ct", "seg"], ensure_channel_first = True),
        SpatialPadd(keys=["ct", "seg"], spatial_size=(200, 200, 310), method='end'),
        Orientationd(keys=["ct",  "seg"], axcodes="PLS"),
        #NormalizeIntensityd(keys=["pt"]),
        ClipCT(keys=["ct"]),
        ScaleIntensityd(keys=["ct"], minv=0, maxv=1),
        #MulPTFM(keys=["ct","pt"]),
        #ConcatItemsd(keys=["pt", "ct"], name="ctpt"),
    ]
)

In [18]:
train_ds = monai.data.Dataset(data=train_files, transform=train_transforms)

train_loader = DataLoader(
    train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=8,
    pin_memory=torch.cuda.is_available(),
    )

val_ds = monai.data.Dataset(data=validation_files, transform=val_transforms)

val_loader = DataLoader(
    val_ds, 
    batch_size=2, 
    num_workers=8, 
    shuffle= False)

In [19]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNETR(
    in_channels=1,
    out_channels=3,
    img_size=(96, 96, 96),
    feature_size=16,
    hidden_size=768,
    mlp_dim=3072, 
    num_heads=12,
    pos_embed="perceptron",
    norm_name="instance",
    res_block=True,
    dropout_rate=0.0,
).to(device)

/home/nada.saadi/miniconda3/envs/clam/lib/python3.8/site-packages/monai/utils/deprecate_utils.py:221: FutureWarning: monai.networks.nets.unetr UNETR.__init__:pos_embed: Argument `pos_embed` has been deprecated since version 1.2. It will be removed in version 1.4. please use `proj_type` instead.
  warn_deprecated(argname, msg, warning_category)


In [20]:
torch.backends.cudnn.benchmark = True
def poly_lr(epoch, max_epochs, initial_lr, exponent=0.9):
    return initial_lr * (1 - epoch / max_epochs)**exponent

loss_function = DiceCELoss(to_onehot_y=True, softmax=True)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)

In [21]:
model_dir = '/home/nada.saadi/CTPET/hecktor2022_cropped/upperboundct'

def validation(epoch_iterator_val):
    model.eval()
    with torch.no_grad():
        for step, batch in enumerate(epoch_iterator_val):
            val_inputs, val_labels = (batch["ct"].cuda(), batch["seg"].cuda())
            val_outputs = sliding_window_inference(val_inputs, (96, 96, 96), 4, model)
            val_labels_list = decollate_batch(val_labels)
            val_labels_convert = [
                post_label(val_label_tensor) for val_label_tensor in val_labels_list
            ]
            val_outputs_list = decollate_batch(val_outputs)
            val_output_convert = [
                post_pred(val_pred_tensor) for val_pred_tensor in val_outputs_list
            ]
            dice_metric(y_pred=val_output_convert, y=val_labels_convert)
            dice_metric_batch(y_pred=val_output_convert, y=val_labels_convert)
            epoch_iterator_val.set_description(
                "Validate (%d / %d Steps)" % (global_step, 10.0)
            )
        mean_dice_val = dice_metric.aggregate().item()
        metric_batch_val = dice_metric_batch.aggregate()

        metric_tumor = metric_batch_val[0].item()
        metric_lymph = metric_batch_val[1].item()

        dice_metric.reset()
        dice_metric_batch.reset()
    return mean_dice_val, metric_tumor, metric_lymph


def train(global_step, train_loader, dice_val_best, global_step_best):
    model.train()
    epoch_loss = 0
    step = 0
    epoch_iterator = tqdm(
        train_loader, desc="Training (X / X Steps) (loss=X.X)", dynamic_ncols=True
    )
    for step, batch in enumerate(epoch_iterator):
        step += 1
        x, y = (batch["ct"].cuda(), batch["seg"].cuda())
        logit_map = model(x)
        loss = loss_function(logit_map, y)
        loss.backward()
        epoch_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()
        epoch_iterator.set_description(
            "Training (%d / %d Steps) (loss=%2.5f)"
            % (global_step, max_iterations, loss)
        )
        if (
            global_step % eval_num == 0 and global_step != 0
        ) or global_step == max_iterations:
            epoch_iterator_val = tqdm(
                val_loader, desc="Validate (X / X Steps) (dice=X.X)", dynamic_ncols=True
            )
            dice_val, metric_tumor, metric_lymph = validation(epoch_iterator_val)
            epoch_loss /= step
            epoch_loss_values.append(epoch_loss)
            metric_values.append(dice_val)
            metric_values_tumor.append(metric_tumor)
            metric_values_lymph.append(metric_lymph)
            if dice_val > dice_val_best:
                dice_val_best = dice_val
                global_step_best = global_step
                torch.save(
                    model.state_dict(), os.path.join(model_dir, "upperboundCT5centers.pth")
                )
                print(
                    "Model Was Saved ! Current Best Avg. Dice: {} Current Avg. Dice: {} Current Avg. tumor Dice: {} Current Avg. lymph Dice: {}".format(
                        dice_val_best, dice_val, metric_tumor, metric_lymph
                    )
                )
            else:
                print(
                    "Model Was Not Saved ! Current Best Avg. Dice: {} Current Avg. Dice: {} Current Avg. tumor Dice: {} Current Avg. lymph Dice: {}".format(
                        dice_val_best, dice_val,  metric_tumor, metric_lymph
                    )
                )
        global_step += 1
    return global_step, dice_val_best, global_step_best


max_iterations = 18000
eval_num = 100

post_label = AsDiscrete(to_onehot=3)
post_pred = AsDiscrete(argmax=True, to_onehot=3)

dice_metric = DiceMetric(include_background=False, reduction="mean", get_not_nans=False)
dice_metric_batch = DiceMetric(include_background=False, reduction="mean_batch")

epoch = 0
max_num_epochs = 530

global_step = 0
dice_val_best = 0.0
global_step_best = 0
epoch_loss_values = []
metric_values = []
metric_values_tumor = []
metric_values_lymph = []
while global_step < max_iterations:
    global_step, dice_val_best, global_step_best = train(
        global_step, train_loader, dice_val_best, global_step_best
    )
    # wandb.log({'learning_rate': optimizer.param_groups[0]['lr']})
    # wandb.log({'Best Dice': dice_val_best})
    epoch += 1
    #optimizer.param_groups[0]['lr'] = poly_lr(epoch, max_num_epochs, 0.005676 , 0.9)
# model.load_state_dict(torch.load(os.path.join(model_dir, "best_metric_luck_UNETr_prompt.pth")))

Training (100 / 18000 Steps) (loss=1.23395):  55%|█████▍    | 101/185 [02:01<25:52, 18.48s/it]

Model Was Saved ! Current Best Avg. Dice: 0.000450061954325065 Current Avg. Dice: 0.000450061954325065 Current Avg. tumor Dice: 0.0006345253787003458 Current Avg. lymph Dice: 0.0


Training (200 / 18000 Steps) (loss=1.12149):   9%|▊         | 16/185 [01:07<49:10, 17.46s/it]

Model Was Not Saved ! Current Best Avg. Dice: 0.000450061954325065 Current Avg. Dice: 4.694758899859153e-05 Current Avg. tumor Dice: 5.956683526164852e-05 Current Avg. lymph Dice: 0.0


Training (300 / 18000 Steps) (loss=1.03511):  63%|██████▎   | 116/185 [02:50<19:23, 16.86s/it]

...

Training (18000 / 18000 Steps) (loss=0.49138):  30%|███       | 56/185 [01:14<31:56, 14.86s/it]

Model Was Not Saved ! Current Best Avg. Dice: 0.4392798840999603 Current Avg. Dice: 0.4294072985649109 Current Avg. tumor Dice: 0.40335556864738464 Current Avg. lymph Dice: 0.4705875813961029


Training (18100 / 18000 Steps) (loss=0.50376):  84%|████████▍ | 156/185 [02:45<07:12, 14.90s/it]

Model Was Not Saved ! Current Best Avg. Dice: 0.4392798840999603 Current Avg. Dice: 0.41742953658103943 Current Avg. tumor Dice: 0.4133160412311554 Current Avg. lymph Dice: 0.426857590675354


Training (18129 / 18000 Steps) (loss=0.58543): 100%|██████████| 185/185 [02:58<00:00,  1.04it/s]


In [ ]:
upper_bound='/home/nada.saadi/CTPET/hecktor2022_cropped/upperboundct/upperboundCT5centers.pth'

In [25]:
import io
from monai.transforms import AsDiscrete
from monai.data import DataLoader, Dataset, decollate_batch
ct_data='/home/nada.saadi/CTPET/hecktor2022_cropped/test_ct_only_mda.json'
with open(ct_data, 'r') as f:
    test_data = json.load(f)

test_loader = DataLoader(
    dataset=Dataset(data=test_data, transform=val_transforms),
    batch_size=1,  # Batch size for testing can be 1 since no backpropagation is required
    shuffle=False,
    num_workers=4
)

loss_function = DiceCELoss(to_onehot_y=True, softmax=True)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)

post_label = AsDiscrete(to_onehot=3)
post_pred = AsDiscrete(argmax=True, to_onehot=3)

# Dice metric for evaluation
dice_metric_fn = DiceMetric(include_background=False, reduction="mean", get_not_nans=False)
dice_metric_batch_fn = DiceMetric(include_background=False, reduction="mean_batch")

#upper_bound='/home/nada.saadi/CTPET/hecktor2022_cropped/upperboundct/upperboundCT5centers.pth'
lower_bound='/home/nada.saadi/CTPET/hecktor2022_cropped/4centers-ctonly/4centers-ctonly.pth'
model.load_state_dict(torch.load(lower_bound))


def testing():
    model.eval()
    dice_metric_fn.reset()
    dice_metric_batch_fn.reset()
    with torch.no_grad():
        for batch_data in tqdm(test_loader):
            test_inputs, test_labels = batch_data["ct"].cuda(), batch_data["seg"].cuda()
            
            test_outputs_ct = sliding_window_inference(test_inputs, (96, 96, 96), 4, model)
            
            # Convert outputs and labels to one-hot format for DiceMetric calculation
            test_outputs_ct = [AsDiscrete(argmax=True, to_onehot=3)(i) for i in decollate_batch(test_outputs_ct)]
            test_labels = [AsDiscrete(to_onehot=3)(i) for i in decollate_batch(test_labels)]

            dice_metric_fn(y_pred=test_outputs_ct, y=test_labels)
            dice_metric_batch_fn(y_pred=test_outputs_ct, y=test_labels)
            
        # After processing all batches, save the outputs
    ct_save_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/test_outputupperbound_ct_saved.pt'
    torch.save(test_outputs_ct, ct_save_path)
    print(f"Outputs saved at: {ct_save_path}")
    

    mean_dice_test = dice_metric_fn.aggregate().item()
    metric_batch_test = dice_metric_batch_fn.aggregate()
    metric_tumor = metric_batch_test[0].item()
    metric_lymph = metric_batch_test[1].item()

    print(f"Testing - Avg Dice: {mean_dice_test:.4f}, Tumor Dice: {metric_tumor:.4f}, Lymph Dice: {metric_lymph:.4f}")
    return mean_dice_test, metric_tumor, metric_lymph
    # Save the weights
testing()

100%|██████████| 44/44 [00:27<00:00,  1.60it/s]


Outputs saved at: /home/nada.saadi/CTPET/hecktor2022_cropped/test_outputupperbound_ct_saved.pt
Testing - Avg Dice: 0.4246, Tumor Dice: 0.3987, Lymph Dice: 0.4688


(0.4245952367782593, 0.39874354004859924, 0.4688272476196289)